<a href="https://colab.research.google.com/github/CiinKRG/RT-card/blob/main/chatbot_bbva.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 47 kB/s 
     |████████████████████████████████| 199 kB 41.3 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=e00f73671a075e66762d019e40b687f1d3d22e3d3fe5ccef6c4cf7d55d4bcad5
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [ ]:
import pyspark 
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder \
   .master("local") \
   .appName("ChatBot BBVA") \
   .config("spark.executor.memory", "6gb") \
   .getOrCreate()

In [ ]:
#Subir en la siguiente linea un csv con el nombre ChatbotBBVA808.csv el cual su información venga de la pestaña chatbot del link
# https://docs.google.com/spreadsheets/d/1MtkuEgolk-y5MYodhveIrxFQqAPptaBZCujOCiJ4qpQ/edit#gid=799512759

In [ ]:
from google.colab import files 
uploaded808 = files.upload()

Saving ChatbotBBVA808.csv to ChatbotBBVA808.csv


In [ ]:
df = spark.read.format('com.databricks.spark.csv')\
                    .options(header='true', inferschema='true')\
                    .option("encoding", "latin1")\
                    .load('ChatbotBBVA808.csv')

In [ ]:
df.show(140)

+---+---------+--------------------+
| id|    label|                chat|
+---+---------+--------------------+
|  1| aceptado|ok acepto la tarjeta|
|  2|rechazado| no estoy interesado|
|  3| aceptado|              acepto|
|  4| aceptado|                  sí|
|  5| aceptado|   quiero la tarjeta|
|  6| aceptado|       ok, yo acepto|
|  7| aceptado|              acepto|
|  8| aceptado|     aceptar tarjeta|
|  9| aceptado|            va va va|
| 10| aceptado|                vale|
| 11| aceptado|                  ok|
| 12| aceptado|          afirmativo|
| 13| aceptado|es correcto, la q...|
| 14| aceptado|   afirmativo colega|
| 15| aceptado|          afirmativo|
| 16| aceptado|          sip acepto|
| 17| aceptado|             siuuuuu|
| 18| aceptado|me gustaria tener...|
| 19| aceptado|      si me gustaria|
| 20| aceptado|           ai acepto|
| 21| aceptado|        vale gracias|
| 22| aceptado|muchas gracias ac...|
| 23| aceptado|      gracias acepto|
| 24| aceptado|     acepto, gracias|
|

In [ ]:
pip install googletrans==4.0.0-rc1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 2.0 MB/s 
     |████████████████████████████████| 42 kB 748 kB/s 
     |████████████████████████████████| 1.4 MB 11.0 MB/s 
     |████████████████████████████████| 53 kB 1.7 MB/s 
     |████████████████████████████████| 65 kB 2.7 MB/s 
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17416 sha256=f37596d34f2baee560e3ea9aad1ca2c7832796746eed5b55c4f56f3c69db62a6
  Stored in directory: /root/.cache/pip/wheels/43/34/00/4fe71786ea6d12314b29037620c36d857e5d104ac2748bf82a
Successfully built googletrans


In [ ]:
from googletrans import Translator
translator = Translator()

In [ ]:
pandasDF = df.toPandas()
pandasDF['translated_label'] = pandasDF['label'].apply(lambda x: translator.translate(x, dest='en').text)
pandasDF['translated_chat'] = pandasDF['chat'].apply(lambda x: translator.translate(x, dest='en').text)
pandasDF = pandasDF.drop(['label', 'chat'], axis=1)

In [ ]:
pandasDF.head()

,id,translated_label,translated_chat
0,1,accepted,OK I accept the card
1,2,refused,I am not interested
2,3,accepted,I agree
3,4,accepted,Yes
4,5,accepted,I want the card


In [ ]:
df = spark.createDataFrame(pandasDF)
df = df.withColumnRenamed("translated_label","label")
df = df.withColumnRenamed("translated_chat","chat")
df.show()

+---+--------+--------------------+
| id|   label|                chat|
+---+--------+--------------------+
|  1|accepted|OK I accept the card|
|  2| refused| I am not interested|
|  3|accepted|             I agree|
|  4|accepted|                 Yes|
|  5|accepted|     I want the card|
|  6|accepted|        ok, I accept|
|  7|accepted|             I agree|
|  8|accepted|         Accept card|
|  9|accepted|                will|
| 10|accepted|                vale|
| 11|accepted|                  ok|
| 12|accepted|         affirmative|
| 13|accepted|It's correct, I w...|
| 14|accepted|affirmative colle...|
| 15|accepted|         affirmative|
| 16|accepted|            I accept|
| 17|accepted|             Suuuuuu|
| 18|accepted|I would like to h...|
| 19|accepted|     yes I'd like to|
| 20|accepted|     You have acepto|
+---+--------+--------------------+
only showing top 20 rows



In [ ]:
df.groupBy("label") \
    .count() \
    .orderBy("count", ascending = False) \
    .show()

+--------+-----+
|   label|count|
+--------+-----+
| refused|   79|
|accepted|   59|
+--------+-----+



In [ ]:
import pyspark.sql.functions as F
df = df.withColumn('word_count',F.size(F.split(F.col('chat'),' ')))

In [ ]:
df.show()

+---+--------+--------------------+----------+
| id|   label|                chat|word_count|
+---+--------+--------------------+----------+
|  1|accepted|OK I accept the card|         5|
|  2| refused| I am not interested|         4|
|  3|accepted|             I agree|         2|
|  4|accepted|                 Yes|         1|
|  5|accepted|     I want the card|         4|
|  6|accepted|        ok, I accept|         3|
|  7|accepted|             I agree|         2|
|  8|accepted|         Accept card|         2|
|  9|accepted|                will|         1|
| 10|accepted|                vale|         1|
| 11|accepted|                  ok|         1|
| 12|accepted|         affirmative|         1|
| 13|accepted|It's correct, I w...|         5|
| 14|accepted|affirmative colle...|         2|
| 15|accepted|         affirmative|         1|
| 16|accepted|            I accept|         2|
| 17|accepted|             Suuuuuu|         1|
| 18|accepted|I would like to h...|         7|
| 19|accepted

In [ ]:
df.groupBy('label')\
    .agg(F.avg('word_count').alias('avg_word_count'))\
    .orderBy('avg_word_count', ascending = False) \
    .show()

+--------+-----------------+
|   label|   avg_word_count|
+--------+-----------------+
| refused| 5.69620253164557|
|accepted|4.813559322033898|
+--------+-----------------+



In [ ]:
!pip3 install textblob

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from textblob import TextBlob
def sentiment_score(chat):
        return TextBlob(chat).sentiment.polarity

In [ ]:
from pyspark.sql.types import FloatType
sentiment_score_udf = F.udf(lambda x: sentiment_score(x), FloatType())

In [ ]:
df = df.select('id', 'label', 'chat','word_count',
                   sentiment_score_udf('chat').alias('sentiment_score'))
df.show()

+---+--------+--------------------+----------+---------------+
| id|   label|                chat|word_count|sentiment_score|
+---+--------+--------------------+----------+---------------+
|  1|accepted|OK I accept the card|         5|            0.5|
|  2| refused| I am not interested|         4|         -0.125|
|  3|accepted|             I agree|         2|            0.0|
|  4|accepted|                 Yes|         1|            0.0|
|  5|accepted|     I want the card|         4|            0.0|
|  6|accepted|        ok, I accept|         3|            0.5|
|  7|accepted|             I agree|         2|            0.0|
|  8|accepted|         Accept card|         2|            0.0|
|  9|accepted|                will|         1|            0.0|
| 10|accepted|                vale|         1|            0.0|
| 11|accepted|                  ok|         1|            0.5|
| 12|accepted|         affirmative|         1|            0.6|
| 13|accepted|It's correct, I w...|         5|         

In [ ]:
df.groupBy('label')\
    .agg(F.avg('sentiment_score').alias('avg_sentiment_score'))\
    .orderBy('avg_sentiment_score', ascending = False) \
    .show()

+--------+--------------------+
|   label| avg_sentiment_score|
+--------+--------------------+
|accepted|  0.1538983058121245|
| refused|0.060213984049196485|
+--------+--------------------+



In [ ]:
df = df.withColumn('words',F.split(F.col('chat'),' '))
df.show()

+---+--------+--------------------+----------+---------------+--------------------+
| id|   label|                chat|word_count|sentiment_score|               words|
+---+--------+--------------------+----------+---------------+--------------------+
|  1|accepted|OK I accept the card|         5|            0.5|[OK, I, accept, t...|
|  2| refused| I am not interested|         4|         -0.125|[I, am, not, inte...|
|  3|accepted|             I agree|         2|            0.0|          [I, agree]|
|  4|accepted|                 Yes|         1|            0.0|               [Yes]|
|  5|accepted|     I want the card|         4|            0.0|[I, want, the, card]|
|  6|accepted|        ok, I accept|         3|            0.5|    [ok,, I, accept]|
|  7|accepted|             I agree|         2|            0.0|          [I, agree]|
|  8|accepted|         Accept card|         2|            0.0|      [Accept, card]|
|  9|accepted|                will|         1|            0.0|              

In [ ]:
stop_words = ['i','me','my','myself','we','our','ours','ourselves',
              'you','your','yours','yourself','yourselves','he','him',
              'his','himself','she','her','hers','herself','it','its',
              'itself','they','them','their','theirs','themselves',
              'what','which','who','whom','this','that','these','those',
              'am','is','are','was','were','be','been','being','have',
              'has','had','having','do','does','did','doing','a','an',
              'the','and','but','if','or','because','as','until','while',
              'of','at','by','for','with','about','against','between',
              'into','through','during','before','after','above','below',
              'to','from','up','down','in','out','on','off','over','under',
              'again','further','then','once','here','there','when','where',
              'why','how','all','any','both','each','few','more','most',
              'other','some','such','no','nor','not','only','own','same',
              'so','than','too','very','can','will','just','don','should','now']

In [ ]:
from pyspark.ml.feature import StopWordsRemover

In [ ]:
stopwordsRemovalFeature = StopWordsRemover(inputCol="words", 
                                           outputCol="words without stop").setStopWords(stop_words)

In [ ]:
from pyspark.ml import Pipeline
stopWordRemovalPipeline = Pipeline(stages=[stopwordsRemovalFeature])
pipelineFitRemoveStopWords = stopWordRemovalPipeline.fit(df)

In [ ]:
df = pipelineFitRemoveStopWords.transform(df)
df.select('words', 'words without stop').show(5)

+--------------------+------------------+
|               words|words without stop|
+--------------------+------------------+
|[OK, I, accept, t...|[OK, accept, card]|
|[I, am, not, inte...|      [interested]|
|          [I, agree]|           [agree]|
|               [Yes]|             [Yes]|
|[I, want, the, card]|      [want, card]|
+--------------------+------------------+
only showing top 5 rows



In [ ]:
df.select('words', 'words without stop').show(140)

+--------------------+--------------------+
|               words|  words without stop|
+--------------------+--------------------+
|[OK, I, accept, t...|  [OK, accept, card]|
|[I, am, not, inte...|        [interested]|
|          [I, agree]|             [agree]|
|               [Yes]|               [Yes]|
|[I, want, the, card]|        [want, card]|
|    [ok,, I, accept]|       [ok,, accept]|
|          [I, agree]|             [agree]|
|      [Accept, card]|      [Accept, card]|
|              [will]|                  []|
|              [vale]|              [vale]|
|                [ok]|                [ok]|
|       [affirmative]|       [affirmative]|
|[It's, correct,, ...|[It's, correct,, ...|
|[affirmative, col...|[affirmative, col...|
|       [affirmative]|       [affirmative]|
|         [I, accept]|            [accept]|
|           [Suuuuuu]|           [Suuuuuu]|
|[I, would, like, ...| [would, like, card]|
|[yes, I'd, like, to]|    [yes, I'd, like]|
| [You, have, acepto]|          

In [ ]:
label = F.udf(lambda x: 1.0 if x == 'accepted' else 0.0, FloatType())
df = df.withColumn('label', label('label'))

In [ ]:
df.select('label').show()

+-----+
|label|
+-----+
|  1.0|
|  0.0|
|  1.0|
|  1.0|
|  1.0|
|  1.0|
|  1.0|
|  1.0|
|  1.0|
|  1.0|
|  1.0|
|  1.0|
|  1.0|
|  1.0|
|  1.0|
|  1.0|
|  1.0|
|  1.0|
|  1.0|
|  1.0|
+-----+
only showing top 20 rows



In [ ]:
import pyspark.ml.feature as feat
TF_ = feat.HashingTF(inputCol="words without stop", 
                     outputCol="rawFeatures", numFeatures=100000)
IDF_ = feat.IDF(inputCol="rawFeatures", outputCol="features")

In [ ]:
pipelineTFIDF = Pipeline(stages=[TF_, IDF_])

In [ ]:
pipelineFit = pipelineTFIDF.fit(df)
df = pipelineFit.transform(df)

In [ ]:
df.select('label', 'rawFeatures','features').show()

+-----+--------------------+--------------------+
|label|         rawFeatures|            features|
+-----+--------------------+--------------------+
|  1.0|(100000,[2144,626...|(100000,[2144,626...|
|  0.0|(100000,[40468],[...|(100000,[40468],[...|
|  1.0|(100000,[46925],[...|(100000,[46925],[...|
|  1.0|(100000,[66161],[...|(100000,[66161],[...|
|  1.0|(100000,[2144,177...|(100000,[2144,177...|
|  1.0|(100000,[52024,62...|(100000,[52024,62...|
|  1.0|(100000,[46925],[...|(100000,[46925],[...|
|  1.0|(100000,[2144,994...|(100000,[2144,994...|
|  1.0|      (100000,[],[])|      (100000,[],[])|
|  1.0|(100000,[23712],[...|(100000,[23712],[...|
|  1.0|(100000,[42645],[...|(100000,[42645],[...|
|  1.0|(100000,[6005],[1...|(100000,[6005],[3...|
|  1.0|(100000,[17712,79...|(100000,[17712,79...|
|  1.0|(100000,[6005,906...|(100000,[6005,906...|
|  1.0|(100000,[6005],[1...|(100000,[6005],[3...|
|  1.0|(100000,[62657],[...|(100000,[62657],[...|
|  1.0|(100000,[25360],[...|(100000,[25360],[...|


In [ ]:
(trainingDF, testDF) = df.randomSplit([0.75, 0.25], seed = 1234)

In [ ]:
from pyspark.ml.classification import LogisticRegression
logreg = LogisticRegression(regParam=0.025)

In [ ]:
logregModel = logreg.fit(trainingDF)


In [ ]:
predictionDF = logregModel.transform(testDF)

In [ ]:
predictionDF.select('label', 'probability', 'prediction').show(140)

+-----+--------------------+----------+
|label|         probability|prediction|
+-----+--------------------+----------+
|  0.0|[0.90332958023517...|       0.0|
|  1.0|[0.65836117838317...|       0.0|
|  1.0|[0.01221630019147...|       1.0|
|  1.0|[0.29778525894469...|       1.0|
|  1.0|[0.02261718638979...|       1.0|
|  1.0|[0.80711865693835...|       0.0|
|  1.0|[0.00831853776327...|       1.0|
|  1.0|[0.64387209426566...|       0.0|
|  1.0|[0.08158025677054...|       1.0|
|  1.0|[0.14717424838074...|       1.0|
|  1.0|[0.02820731959989...|       1.0|
|  1.0|[0.03817921487397...|       1.0|
|  1.0|[0.01854557370396...|       1.0|
|  0.0|[0.28145115834333...|       1.0|
|  0.0|[0.97463381931330...|       0.0|
|  0.0|[0.64387209426566...|       0.0|
|  0.0|[0.73110703010859...|       0.0|
|  0.0|[0.98586982983375...|       0.0|
|  0.0|[0.91443966044666...|       0.0|
|  0.0|[0.64387209426566...|       0.0|
|  0.0|[0.24382097168818...|       1.0|
|  0.0|[0.59039603518640...|       0.0|


In [ ]:
predictionDF.crosstab('label', 'prediction').show()

+----------------+---+---+
|label_prediction|0.0|1.0|
+----------------+---+---+
|             1.0|  4| 10|
|             0.0| 25|  3|
+----------------+---+---+



In [ ]:
from sklearn import metrics
actual = predictionDF.select('label').toPandas()
predicted = predictionDF.select('prediction').toPandas()

In [ ]:
print('accuracy score: {}%'.format(round(metrics.accuracy_score(actual, predicted),3)*100))

accuracy score: 83.3%


In [ ]:
# Tenemos un 83.3 % de efectividad en saber si el cliente querrá la tarjeta de credito o no con base en la
# medición del sentimiento de su respuesta

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

scores = predictionDF.select('label', 'rawPrediction')
evaluator = BinaryClassificationEvaluator()
print('El score ROC score es {}%'.format(round(evaluator.evaluate(scores),3)*100))

El score ROC score es 91.3%
